In [107]:
#imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [108]:
df = pd.read_csv('traindata2.csv')
df.head()

,redCorner,blueCorner,winner,referee,venue,title_fight,billing,redCorner_wins,blueCorner_wins,redCorner_losses,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,2,48,193,0,0,12,12,2,...,0,1.360000,4.480000,25.000000,4.480000,1.360000,0.407407,0.776316,1.0,1.000000
1,0,2,0,82,193,0,1,17,27,5,...,1,1.533333,1.533333,15.000000,1.533333,1.533333,0.610169,0.452381,0.5,0.500000
2,0,2,0,66,193,0,2,17,19,5,...,0,3.959391,3.045685,3.283333,3.045685,3.959391,0.545455,0.717391,1.0,1.000000
3,0,2,2,82,193,0,2,12,16,5,...,1,1.733333,2.533333,15.000000,2.533333,1.733333,0.703125,0.611940,1.0,0.888889
4,0,2,1,48,193,0,2,14,26,0,...,0,5.454545,5.454545,0.183333,5.454545,5.454545,0.000000,0.500000,1.0,1.000000


In [109]:
dfInput = pd.read_csv('customInput.csv')
dfInput = dfInput.drop('Unnamed: 0', axis=1, errors='ignore')
dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,redCorner_fightTime,blueCorner_fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,Merab Dvalishvili,Henry Cejudo,NaN,16,16,4,3,0,0,33.0,...,4.46,3.93,15.4,13.35,2.41,3.15,0.56,0.6,0.78,0.83


In [110]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6155
6155


In [111]:
#dropping referee
df.drop('referee', axis=1, inplace=True)

#drop billings
df.drop('billing', axis=1, inplace=True)

#drop venue
df.drop('venue', axis=1, inplace=True)

#drop title_fight
df.drop('title_fight', axis=1, inplace=True)


df.head()


,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_stance,redCorner_sig_str_landed_per_minute,blueCorner_sig_str_landed_per_minute,fightTime,redCorner_sig_str_absorbed_per_minute,blueCorner_sig_str_absorbed_per_minute,redCorner_sig_str_defense,blueCorner_sig_str_defense,redCorner_takedown_defense,blueCorner_takedown_defense
0,0,2,2,12,12,2,4,0,0,35.0,...,0,1.360000,4.480000,25.000000,4.480000,1.360000,0.407407,0.776316,1.0,1.000000
1,0,2,0,17,27,5,12,1,0,34.0,...,1,1.533333,1.533333,15.000000,1.533333,1.533333,0.610169,0.452381,0.5,0.500000
2,0,2,0,17,19,5,4,0,0,33.0,...,0,3.959391,3.045685,3.283333,3.045685,3.959391,0.545455,0.717391,1.0,1.000000
3,0,2,2,12,16,5,5,0,1,37.0,...,1,1.733333,2.533333,15.000000,2.533333,1.733333,0.703125,0.611940,1.0,0.888889
4,0,2,1,14,26,0,8,0,0,33.0,...,0,5.454545,5.454545,0.183333,5.454545,5.454545,0.000000,0.500000,1.0,1.000000


In [112]:
df.rename(columns={'fightTime': 'redCorner_fightTime'}, inplace=True)
df.insert(32, 'blueCorner_fightTime', df['redCorner_fightTime'])

dfColumns = df.columns
print(len(dfColumns))

39


In [113]:
dfInputColumns = dfInput.columns
print(len(dfInputColumns))

39


In [114]:
#one hot encode redCorner_nation
column = df[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = df[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_101,blueCorner_nation_102,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_109,blueCorner_nation_110
0,0,2,2,12,12,2,4,0,0,35.0,...,0,0,0,0,0,0,0,0,0,0
1,0,2,0,17,27,5,12,1,0,34.0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,17,19,5,4,0,0,33.0,...,0,0,0,0,0,0,0,0,0,0
3,0,2,2,12,16,5,5,0,1,37.0,...,0,0,0,0,0,0,0,0,0,0
4,0,2,1,14,26,0,8,0,0,33.0,...,0,0,0,0,0,0,0,0,0,0


In [115]:
#one hot encode redCorner_stance
column = df[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
df = pd.concat([df, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = df[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
df = pd.concat([df, df_encoded], axis=1)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_109,blueCorner_nation_110,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,2,2,12,12,2,4,0,0,35.0,...,0,0,1,0,0,0,1,0,0,0
1,0,2,0,17,27,5,12,1,0,34.0,...,0,0,1,0,0,0,0,1,0,0
2,0,2,0,17,19,5,4,0,0,33.0,...,0,0,1,0,0,0,1,0,0,0
3,0,2,2,12,16,5,5,0,1,37.0,...,0,0,1,0,0,0,0,1,0,0
4,0,2,1,14,26,0,8,0,0,33.0,...,0,0,0,1,0,0,1,0,0,0


In [116]:
for index, row in df.iterrows():
    if row['winner'] == 1:
        df.drop(index)

df['blueCorner'] = 1
df['winner'] = df['winner'].replace(2, 1)


df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_109,blueCorner_nation_110,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,1,1,12,12,2,4,0,0,35.0,...,0,0,1,0,0,0,1,0,0,0
1,0,1,0,17,27,5,12,1,0,34.0,...,0,0,1,0,0,0,0,1,0,0
2,0,1,0,17,19,5,4,0,0,33.0,...,0,0,1,0,0,0,1,0,0,0
3,0,1,1,12,16,5,5,0,1,37.0,...,0,0,1,0,0,0,0,1,0,0
4,0,1,1,14,26,0,8,0,0,33.0,...,0,0,0,1,0,0,1,0,0,0


In [117]:
#one hot encode redCorner_nation
column = df[['redCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_nation'], prefix='redCorner_nation').astype(int)
dfInput.drop('redCorner_nation', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

#one hot encode blueCorner_nation
column = df[['blueCorner_nation']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_nation'], prefix='blueCorner_nation').astype(int)
dfInput.drop('blueCorner_nation', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

df.drop('redCorner_nation', axis=1, inplace=True)
df.drop('blueCorner_nation', axis=1, inplace=True)

dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_101,blueCorner_nation_102,blueCorner_nation_103,blueCorner_nation_104,blueCorner_nation_105,blueCorner_nation_106,blueCorner_nation_107,blueCorner_nation_108,blueCorner_nation_109,blueCorner_nation_110
0,Merab Dvalishvili,Henry Cejudo,NaN,16.0,16.0,4.0,3.0,0.0,0.0,33.0,...,0,0,0,0,0,0,0,0,0,0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0


In [118]:
#one hot encode redCorner_stance
column = df[['redCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['redCorner_stance'], prefix='redCorner_stance').astype(int)
dfInput.drop('redCorner_stance', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

#one hot encode blueCorner_stance
column = df[['blueCorner_stance']].copy()
df_encoded = pd.get_dummies(column, columns=['blueCorner_stance'], prefix='blueCorner_stance').astype(int)
dfInput.drop('blueCorner_stance', axis=1, inplace=True)
dfInput = pd.concat([dfInput, df_encoded], axis=1)

df.drop('redCorner_stance', axis=1, inplace=True)
df.drop('blueCorner_stance', axis=1, inplace=True)

df.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_109,blueCorner_nation_110,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0,1,1,12,12,2,4,0,0,35.0,...,0,0,1,0,0,0,1,0,0,0
1,0,1,0,17,27,5,12,1,0,34.0,...,0,0,1,0,0,0,0,1,0,0
2,0,1,0,17,19,5,4,0,0,33.0,...,0,0,1,0,0,0,1,0,0,0
3,0,1,1,12,16,5,5,0,1,37.0,...,0,0,1,0,0,0,0,1,0,0
4,0,1,1,14,26,0,8,0,0,33.0,...,0,0,0,1,0,0,1,0,0,0


In [121]:
#limit the data
dfInput = dfInput.head(1)
print(dfInput.iloc[0])

redCorner              Merab Dvalishvili
blueCorner                  Henry Cejudo
winner                               NaN
redCorner_wins                      16.0
blueCorner_wins                     16.0
                             ...        
redCorner_stance_3                     0
blueCorner_stance_0                    1
blueCorner_stance_1                    0
blueCorner_stance_2                    0
blueCorner_stance_3                    0
Name: 0, Length: 249, dtype: object


In [122]:
dfColumns = df.columns
for column in dfColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0
redCorner_nation_1
redCorner_nation_2
redCorner_nation_3
redCorner_nation_4
redCorner_nation_5
redCorner_nation_6
redCorner_nation_7
redCorner_nation_8
redCorner_nation_9
redCorner_nation_10
redCorner_na

In [123]:
dfInputColumns = dfInput.columns
for column in dfInputColumns:
    print(column)

redCorner
blueCorner
winner
redCorner_wins
blueCorner_wins
redCorner_losses
blueCorner_losses
redCorner_draws
blueCorner_draws
redCorner_age
blueCorner_age
redCorner_knockdowns
blueCorner_knockdowns
redCorner_sig_str_percentage
blueCorner_sig_str_percentage
redCorner_takedowns
blueCorner_takedowns
redCorner_takedown_percentage
blueCorner_takedown_percentage
redCorner_subs_attempted
blueCorner_subs_attempted
redCorner_height
blueCorner_height
redCorner_reach
blueCorner_reach
redCorner_sig_str_landed_per_minute
blueCorner_sig_str_landed_per_minute
redCorner_fightTime
blueCorner_fightTime
redCorner_sig_str_absorbed_per_minute
blueCorner_sig_str_absorbed_per_minute
redCorner_sig_str_defense
blueCorner_sig_str_defense
redCorner_takedown_defense
blueCorner_takedown_defense
redCorner_nation_0
redCorner_nation_1
redCorner_nation_2
redCorner_nation_3
redCorner_nation_4
redCorner_nation_5
redCorner_nation_6
redCorner_nation_7
redCorner_nation_8
redCorner_nation_9
redCorner_nation_10
redCorner_na

In [124]:
#when one hot encoding, nations that have no representation in the train set have Nan values. This sets them to zero as they would have been zero if they were represented
df.fillna(0, inplace=True)

In [125]:
print(len(df))
df = df.dropna(inplace=False)
print(len(df))

6155
6155


In [126]:
print(len(dfInput))
dfInput.dropna(subset=dfInput.columns.difference(['winner']), inplace=True)
print(len(dfInput))

1
1


In [127]:
dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_109,blueCorner_nation_110,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,Merab Dvalishvili,Henry Cejudo,NaN,16.0,16.0,4.0,3.0,0.0,0.0,33.0,...,0,0,1,0,0,0,1,0,0,0


In [128]:
#x,y split
target_column = 'winner'
y = df[target_column]
X = df.drop(target_column, axis=1)

#create train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)





In [136]:
#Define the deep learning model
model = Sequential()
model.add(Dense(64, activation = 'relu'))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

#earlyStopping
earlystopping = EarlyStopping(monitor='val_loss', patience=100, restore_best_weights=True)

# Define the ModelCheckpoint callback
checkpoint = ModelCheckpoint(filepath='best_model_weights2.h5', save_weights_only=True, monitor='val_accuracy', save_best_only=True, verbose=1)

#compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#train
model.fit(X_train, y_train, epochs=1000, batch_size=124, validation_data=(X_test, y_test), callbacks=[checkpoint, earlystopping])

Epoch 1/1000
38/40 [===========================>..] - ETA: 0s - loss: 1.0474 - accuracy: 0.5199
Epoch 1: val_accuracy improved from -inf to 0.58814, saving model to best_model_weights2.h5
40/40 [==============================] - 2s 9ms/step - loss: 1.0313 - accuracy: 0.5223 - val_loss: 0.6683 - val_accuracy: 0.5881
Epoch 2/1000
40/40 [==============================] - ETA: 0s - loss: 0.6149 - accuracy: 0.6602
Epoch 2: val_accuracy improved from 0.58814 to 0.78310, saving model to best_model_weights2.h5
40/40 [==============================] - 0s 9ms/step - loss: 0.6149 - accuracy: 0.6602 - val_loss: 0.5282 - val_accuracy: 0.7831
Epoch 3/1000
30/40 [=====================>........] - ETA: 0s - loss: 0.4753 - accuracy: 0.8027
Epoch 3: val_accuracy improved from 0.78310 to 0.81966, saving model to best_model_weights2.h5
40/40 [==============================] - 0s 5ms/step - loss: 0.4684 - accuracy: 0.8018 - val_loss: 0.4228 - val_accuracy: 0.8197
Epoch 4/1000
35/40 [=======================

In [137]:
for index, row in dfInput.iterrows():
    dfInput.at[index, 'redCorner'] = 0.0
    dfInput.at[index, 'blueCorner'] = 1.0

dfInput.head()

,redCorner,blueCorner,winner,redCorner_wins,blueCorner_wins,redCorner_losses,blueCorner_losses,redCorner_draws,blueCorner_draws,redCorner_age,...,blueCorner_nation_109,blueCorner_nation_110,redCorner_stance_0,redCorner_stance_1,redCorner_stance_2,redCorner_stance_3,blueCorner_stance_0,blueCorner_stance_1,blueCorner_stance_2,blueCorner_stance_3
0,0.0,1.0,NaN,16.0,16.0,4.0,3.0,0.0,0.0,33.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [138]:
dfInput = dfInput.astype(float)

In [139]:
#accuracy
y_hat = model.predict(X_test)
y_hat = [0 if val<0.5 else 1 for val in y_hat]
accuracy_score(y_test,y_hat)

#0.8545897644191714

39/39 [==============================] - 0s 3ms/step


0.8602761982128351

In [140]:
#testing the model using career statistics

# x, y split for dfMock
target_column = 'winner'
y_mock = dfInput[target_column]
X_mock = dfInput.drop(target_column, axis=1)

#accuracy
y_hat = model.predict(X_mock)
print(y_hat)




1/1 [==============================] - 0s 41ms/step
[[0.09351348]]
